# The `Particle` Class

The `Particle` class is the base class for all particles, whether introduced discretely one by one or as a distribution. In reality, the `Particle` class is based on two intermediate classes: `ParticleDistribution` and `ParticleInstances` to instantiate particle distributions and particles directly, respectively.

## The `ParticleDistribution` Class

| attribute    | unit        | default value
| ---------    | ----        | -------------
| `spacing`    |             | `"linspace"`
| `nbins`      |             | `1e3`
| `nparticles` |             | `1e5`
| `volume`     | / m^3       | `1e-6`
| `cutoff`     |             | `0.9999`
| `gsigma`     |             | `1.25`
| `mode`       | m           | `1e-7`

methods:
- pre_radius
- pre_discretize
- pre_distribution

## The `ParticleInstances` Class

| attribute         | unit        | default value
| ---------         | ----        | -------------
| `particle_radius` | m           | `None` or `ParticleDistribution.pre_radius()`
| `particle_number` |             | `1` or `ParticleDistribution.nparticles`
| `particle_density`| kg / m^-3   | `1e3`
| `shape_factor`    |             | `1`
| `volume_void`     |             | `0`
| `particle_charge` |             | `0`


methods:
- particle_distribution
- mass
- knudsen_number
- slip_correction_factor
- friction_factor

## The `Particle` Class

| attribute                   | unit               | default value
| ---------                   | ----               | -------------
| `elementary_charge_value`   | C                  | `constants.ELEMENTARY_CHARGE_VALUE`
| `electric_permittivity`     | F / m              | `constants.ELECTRIC_PERMITTIVITY`
| `boltzmann_constant`        | kg * m^2 / K / s^2 | `constants.BOLTZMANN_CONSTANT`
| `coagulation_approximation` |                    | `"hardsphere"`


methods:
- _coag_prep
- reduced_mass
- reduced_friction_factor
- coulomb_potential_ratio
- coulomb_enhancement_kinetic_limit
- coulomb_enhancement_continuum_limit
- diffusive_knudsen_number
- dimensionless_coagulation
- coagulation